In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import random
import math
import time
from sklearn.linear_model import LinearRegression,BayesianRidge
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,mean_absolute_error
import datetime
import operator
plt.style.use('seaborn')
%matplotlib inline

In [13]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/10-09-2020.csv')
us_medical_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/10-09-2020.csv')
apple_mobility = pd.read_csv('https://covid19-static.cdn-apple.com/covid19-mobility-data/2018HotfixDev22/v3/en-us/applemobilitytrends-2020-10-09.csv')


/Users/jihyechoi/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39285,39290,39297,39341,39422,39486,39548,39616,39693,39703
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,13806,13965,14117,14266,14410,14568,14730,14899,15066,15231
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,51690,51847,51995,52136,52270,52399,52520,52658,52804,52940
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,2050,2110,2110,2110,2370,2370,2568,2568,2696,2696
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,5114,5211,5370,5402,5530,5725,5725,5958,6031,6246


In [15]:
columns=confirmed_df.keys()

In [16]:
confirmed=confirmed_df.loc[:,columns[4]:columns[-1]]
deaths=death_df.loc[:,columns[4]:columns[-1]]
recovered=recovered_df.loc[:,columns[4]:columns[-1]]

In [17]:
dates=confirmed.keys()
world_cases=[]
total_deaths=[]
mortality_rate=[]
total_recovered=[]
for i in dates:
    confirmed_sum=confirmed[i].sum()
    death_sum=deaths[i].sum()
    recovered_sum=recovered[i].sum()
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    mortality_rate.append(death_sum/confirmed_sum)
    total_recovered.append(recovered_sum)

In [18]:
print (total_deaths)

[17, 18, 26, 42, 56, 82, 131, 133, 171, 213, 259, 362, 426, 492, 564, 634, 719, 806, 906, 1013, 1113, 1118, 1371, 1523, 1666, 1770, 1868, 2008, 2123, 2248, 2252, 2459, 2470, 2630, 2710, 2771, 2814, 2873, 2942, 2996, 3085, 3160, 3255, 3348, 3460, 3559, 3803, 3987, 4267, 4611, 4917, 5414, 5834, 6475, 7153, 7964, 8867, 9981, 11460, 13180, 14854, 16797, 19079, 21877, 24893, 28388, 32102, 35572, 39620, 44365, 49863, 56101, 62205, 68351, 73540, 79481, 87712, 94464, 102237, 109629, 115854, 121671, 127587, 134567, 142918, 150221, 158588, 164601, 169862, 175726, 182917, 189644, 196485, 203142, 208774, 212708, 217382, 223929, 230657, 236559, 241722, 247205, 250694, 254808, 260714, 267277, 272579, 278054, 282287, 285874, 289354, 294889, 299981, 305171, 310304, 314424, 317712, 321412, 326158, 330926, 335657, 340851, 344742, 347870, 349062, 353180, 358326, 362944, 367536, 371610, 374478, 377554, 382296, 387745, 392879, 397453, 401265, 404006, 407707, 412562, 417623, 422322, 426545, 430722, 434129, 

In [19]:
days_1_22=np.array([i for i in range(len(dates))]).reshape(-1,1)
world_cases=np.array(world_cases).reshape(-1,1)
total_deaths=np.array(total_deaths).reshape(-1,1)
total_recovered=np.array(total_recovered).reshape(-1,1)

In [20]:
days_future=30
future_forcast=np.array([i for i in range(len(dates)+days_future)]).reshape(-1,1)
adjust_dates=future_forcast[:-30]

In [21]:
start='1/22/2020'
start_date=datetime.datetime.strptime(start,'%m/%d/%Y')
future_forcast_dates=[]
for i in range(len(future_forcast)):
   future_forcast_dates.append((start_date+datetime.timedelta(days=i)).strftime('%m/%d/%Y')) 


In [22]:
x_train_confirmed,x_test_confirrmed,y_train_confirmed,y_test_confirmed=train_test_split(days_1_22,world_cases,test_size=0.15,shuffle=False)

In [ ]:
kernel=['poly','sigmoid','rbf']
c=[0.01,0.1,1,10]
gamma=[0.01,0.1,1]
epsilon=[0.01,0.1,1]
shrinking=[True,False]
svm_grid={'kernel':kernel,'C':c,'gamma':gamma,'epsilon':epsilon,'shrinking':shrinking}
svm=SVR()
svm_search=RandomizedSearchCV(svm,svm_grid,scoring='neg_mean_squared_error',cv=3,return_train_score=True,n_jobs=-1,n_iter=40,verbose=1)
svm_search.fit(x_train_confirmed,y_train_confirmed)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.9s


In [ ]:
svm_confirmed=svm_search.best_estimator_
svm_pred=svm_confirmed.predict(future_forcast)

In [ ]:
linear_model=LinearRegression(normalize=True,fit_intercept=True)
linear_model.fit(x_train_confirmed,y_train_confirmed)
test_linear_pred=linear_model.predict(x_test_confirrmed)
linear_pred=linear_model.predict(future_forcast)
plt.plot(test_linear_pred)
plt.plot(y_test_confirmed)
print('MAE:',mean_absolute_error(test_linear_pred,y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred,y_test_confirmed))

In [ ]:
print(linear_model.coef_)
print(linear_model.intercept_)

In [ ]:
plt.figure(figsize=(20,12))
plt.plot(adjust_dates,world_cases)
plt.plot(future_forcast,linear_pred,linestyle='dashed',color='green')
plt.title('no. of Coronavirus cases over Time',size=30)
plt.xlabel('Day Since 1/22/2020',size=30)
plt.ylabel('no of Cases',size=30)
plt.legend(['Confirmed cases','Linear Regression Pred'])
plt.xticks(size=15)
plt.show()